In [14]:
x = 5 
for i in range(10):
    print (i, (5**i) % 21, (5**i) & 20)

0 1 0
1 5 4
2 4 16
3 20 20
4 16 16
5 17 20
6 1 0
7 5 4
8 4 0
9 20 4


In [103]:
from qiskit import *
import numpy as np
import math 
from qiskit.circuit.library import PhaseEstimation

N = 10
R1 = 8
R2 = 4

QC = QuantumCircuit(R1+R2,R1)

In [104]:
def c_amod15(a, power):
    """Controlled multiplication by a mod 15"""
    if a not in [2,4,7,8,11,13]:
        raise ValueError("'a' must be 2,4,7,8,11 or 13")
    U = QuantumCircuit(4)        
    for iteration in range(power):
        if a in [2,13]:
            U.swap(0,1)
            U.swap(1,2)
            U.swap(2,3)
        if a in [7,8]:
            U.swap(2,3)
            U.swap(1,2)
            U.swap(0,1)
        if a in [4, 11]:
            U.swap(1,3)
            U.swap(0,2)
        if a in [7,11,13]:
            for i in range(4):
                U.x(i)
    U = U.to_gate()
    U.name = "%i^%i mod 15" % (a, power)
    return U

In [105]:
QC.x(-1)
QC.barrier()

PE = PhaseEstimation(R1, c_amod15(7, 1), QFT(num_qubits=R1, inverse=True, do_swaps=True))
QC.append(PE, list(range(R1+R2)))

QC = QC.decompose().decompose()

for i in range(R1):
    QC.measure(i,i)

QC.draw(fold=1000, scale=0.5)

░ ┌───┐                                                                                                                                             ┌──────────┐┌─┐                     
 q_0: ─────────────░─┤ H ├────────■────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤0         ├┤M├─────────────────────
                   ░ ├───┤        │                                                                                                                                    │          │└╥┘┌─┐                  
 q_1: ─────────────░─┤ H ├────────┼────────────────■───────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤1         ├─╫─┤M├──────────────────
                   ░ ├───┤        │                │                                                                                                                   │          │ ║ └╥┘┌─┐               
 q_2: ─────────────░─┤ H ├────────┼────────────────┼────────────────■──────────────────────────────────────────────────────────────────────────────────────────────────┤2         ├─╫──╫─┤M├───────────────
                   ░ ├───┤        │                │                │                                                                                                  │          │ ║  ║ └╥┘┌─┐            
 q_3: ─────────────░─┤ H ├────────┼────────────────┼────────────────┼────────────────■─────────────────────────────────────────────────────────────────────────────────┤3         ├─╫──╫──╫─┤M├────────────
                   ░ ├───┤        │                │                │                │                                                                                 │  IQFT_dg │ ║  ║  ║ └╥┘┌─┐         
 q_4: ─────────────░─┤ H ├────────┼────────────────┼────────────────┼────────────────┼────────────────■────────────────────────────────────────────────────────────────┤4         ├─╫──╫──╫──╫─┤M├─────────
                   ░ ├───┤        │                │                │                │                │                                                                │          │ ║  ║  ║  ║ └╥┘┌─┐      
 q_5: ─────────────░─┤ H ├────────┼────────────────┼────────────────┼────────────────┼────────────────┼─────────────────■──────────────────────────────────────────────┤5         ├─╫──╫──╫──╫──╫─┤M├──────
                   ░ ├───┤        │                │                │                │                │                 │                                              │          │ ║  ║  ║  ║  ║ └╥┘┌─┐   
 q_6: ─────────────░─┤ H ├────────┼────────────────┼────────────────┼────────────────┼────────────────┼─────────────────┼─────────────────■────────────────────────────┤6         ├─╫──╫──╫──╫──╫──╫─┤M├───
                   ░ ├───┤        │                │                │                │                │                 │                 │                            │          │ ║  ║  ║  ║  ║  ║ └╥┘┌─┐
 q_7: ─────────────░─┤ H ├────────┼────────────────┼────────────────┼────────────────┼────────────────┼─────────────────┼─────────────────┼──────────────────■─────────┤7         ├─╫──╫──╫──╫──╫──╫──╫─┤M├
                   ░ └───┘┌───────┴───────┐┌───────┴───────┐┌───────┴───────┐┌───────┴───────┐┌───────┴────────┐┌───────┴────────┐┌───────┴────────┐┌────────┴────────┐└──────────┘ ║  ║  ║  ║  ║  ║  ║ └╥┘
 q_8: ─────────────░──────┤0              ├┤0              ├┤0              ├┤0              ├┤0               ├┤0               ├┤0               ├┤0                ├─────────────╫──╫──╫──╫──╫──╫──╫──╫─
                   ░      │               ││               ││               ││               ││                ││                ││                ││                 │             ║  ║  ║  ║  ║  ║  ║  ║ 
 q_9: ─────────────░──────┤1              ├┤1              ├┤1              ├┤1              ├┤1               ├┤1               ├┤1           

In [106]:
from qiskit.primitives import Sampler

from fractions import Fraction
from math import gcd

a = 8
N = 15

FACTOR_FOUND = False
ATTEMPT = 0
while not FACTOR_FOUND:
    ATTEMPT += 1
    print(f"\nAttempt {ATTEMPT}")
    
    phase = Sampler().run(QC, shots=1).result()
    print (phase)
    phase = phase.quasi_dists[0].popitem()[0]
    phase /= 2**8

    frac = Fraction(phase).limit_denominator(N)
    r = frac.denominator
    if phase != 0:
        # Guess for a factor is gcd(x^{r/2} - 1 , 15)
        guess = gcd(a**(r//2)-1, N)
        if guess not in [1,N] and (N % guess) == 0:
            # Guess is a factor!
            print(f"Non-trivial factor found: {guess}")
            FACTOR_FOUND = True   


Attempt 1
SamplerResult(quasi_dists=[{128: 1.0}], metadata=[{'shots': 1}])

Attempt 2
SamplerResult(quasi_dists=[{64: 1.0}], metadata=[{'shots': 1}])
Non-trivial factor found: 3
